[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slitvinov/dcomex-framework/blob/master/notebook/analysis.ipynb)

In [ ]:
%%sh
if ! test -f .clone
then git clone --depth 1 --quiet https://github.com/slitvinov/dcomex-framework &&
     : > .clone
fi

In [ ]:
import glob
import math
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
import tarfile
import re
import numpy as np

In [ ]:
def plot(*arg, **karg):
    plt.yscale("log")
    plt.xlabel("time (days)")
    plt.ylabel("relative tumor volume (log scale)")
    plt.plot(*arg, **karg)
KEYS = set(("includeImmuno", "k1", "kd", "keepLogs", "k_on", "K_T",
            "k_th_tumor", "location", "location_2", "location_3", "Lp",
            "miTumor", "Per", "pv", "sf", "Sv"))
def read11(path):
    with tarfile.open(path, 'r') as tar:
        for member in tar.getmembers():
            if re.match("^[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]/status$",
                        member.name):
                dirname = re.sub("/status$", "", member.name)
                root = ET.parse(tar.extractfile(dirname + "/MSolveInput.xml"))
                params = {
                    key.tag:
                    True if key.text == "true" else
                    False if key.text == "false" else float(key.text)
                    for key in root.find('./Parameters') if key.tag in KEYS
                }
                status = int(
                    tar.extractfile(dirname +
                                    "/status").read().decode('utf-8'))
                volume_file = tar.extractfile(
                    dirname +
                    "/tumorVolume_AnalysisNo_1.txt").read().decode('utf-8')
                time_file = tar.extractfile(
                    dirname +
                    "/timeStepTotalTimes_AnalysisNo_1.txt").read().decode(
                        'utf-8')
                volume = []
                time = []
                for t, v in zip(time_file.split(), volume_file.split()):
                    if v == "0.000000E+000":
                        break
                    time.append(float(t))
                    volume.append(float(v))
                yield params, time, volume, status


In [ ]:
cnt = 0
for params, time, volume, status in read11("dcomex-framework/notebook/1.tar.gz"):
    if status == 0:
        time = np.divide(time, 60 * 60 * 24)
        volume = np.divide(volume, volume[0])
        location = params["location"]
        index = np.searchsorted(time, location)
        plot(time,
             volume,
             'k-', [location], [volume[index]],
             'ok',
             alpha=0.5)
        cnt += 1
    if cnt == 10:
        break